In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Bước 1: Định nghĩa Mô hình Pydantic cho Kết quả Có cấu trúc

Các mô hình này định nghĩa **cấu trúc dữ liệu** mà các tác nhân sẽ trả về. Việc sử dụng `response_format` với Pydantic đảm bảo:
- ✅ Trích xuất dữ liệu an toàn kiểu
- ✅ Tự động xác thực
- ✅ Không lỗi phân tích từ phản hồi dạng văn bản tự do
- ✅ Dễ dàng chuyển hướng có điều kiện dựa trên các trường dữ liệu


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: Tạo Công Cụ Đặt Phòng Khách Sạn

Công cụ này là thứ mà **availability_agent** sẽ gọi để kiểm tra xem phòng còn trống hay không. Chúng tôi sử dụng trình trang trí `@ai_function` để:
- Chuyển một hàm Python thành công cụ có thể gọi từ AI
- Tự động tạo sơ đồ JSON cho LLM
- Xử lý xác thực tham số
- Kích hoạt tự động gọi bởi các agent

Cho bản demo này:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Có phòng ✅
- **Tất cả các thành phố khác** → Không có phòng ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 3: Định nghĩa các hàm điều kiện cho việc định tuyến

Các hàm này sẽ kiểm tra phản hồi của agent và xác định đường đi nào sẽ được chọn trong quy trình làm việc.

**Mẫu chính:**
1. Kiểm tra xem tin nhắn có phải là `AgentExecutorResponse` không
2. Phân tích đầu ra cấu trúc (mô hình Pydantic)
3. Trả về `True` hoặc `False` để điều khiển định tuyến

Quy trình làm việc sẽ đánh giá các điều kiện này trên **các cạnh** để quyết định executor nào sẽ được gọi tiếp theo.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Bước 4: Tạo Trình Thực Thi Hiển Thị Tùy Chỉnh

Executors là các thành phần của quy trình làm việc thực hiện các biến đổi hoặc hiệu ứng phụ. Chúng ta sử dụng decorator `@executor` để tạo một executor tùy chỉnh hiển thị kết quả cuối cùng.

**Khái niệm chính:**
- `@executor(id="...")` - Đăng ký một hàm như một executor của quy trình làm việc
- `WorkflowContext[Never, str]` - Kiểu gợi ý cho đầu vào/đầu ra
- `ctx.yield_output(...)` - Trả về kết quả cuối cùng của quy trình làm việc


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Bước 5: Tải Biến Môi Trường

Cấu hình client LLM. Ví dụ này hoạt động với:
- **Mô hình GitHub** (Miễn phí với token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Bước 6: Tạo các Đại lý AI với Đầu ra Cấu trúc

Chúng tôi tạo **ba đại lý chuyên biệt**, mỗi đại lý được bao bọc trong `AgentExecutor`:

1. **availability_agent** - Kiểm tra tính khả dụng của khách sạn bằng công cụ
2. **alternative_agent** - Gợi ý các thành phố thay thế (khi không có phòng)
3. **booking_agent** - Khuyến khích đặt phòng (khi có phòng)

**Tính năng chính:**
- `tools=[hotel_booking]` - Cung cấp công cụ cho đại lý
- `response_format=PydanticModel` - Ép đầu ra JSON có cấu trúc
- `AgentExecutor(..., id="...")` - Bao gói đại lý để sử dụng trong quy trình công việc


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Bước 7: Xây dựng Quy trình Công việc với Các Cạnh Điều kiện

Bây giờ chúng ta sử dụng `WorkflowBuilder` để xây dựng đồ thị với định tuyến có điều kiện:

**Cấu trúc Quy trình Công việc:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Các Phương thức Chính:**
- `.set_start_executor(...)` - Đặt điểm bắt đầu
- `.add_edge(from, to, condition=...)` - Thêm cạnh có điều kiện
- `.build()` - Hoàn tất quy trình công việc


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Bước 8: Chạy Trường Hợp Kiểm Tra 1 - Thành Phố KHÔNG có Phòng Trống (Paris)

Hãy kiểm tra trường hợp **không có phòng trống** bằng cách yêu cầu khách sạn ở Paris (nơi không có phòng nào trong mô phỏng của chúng ta).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Bước 9: Chạy Test Case 2 - Thành phố CÓ Sẵn hàng (Stockholm)

Bây giờ hãy kiểm tra đường dẫn **availability** bằng cách yêu cầu các khách sạn ở Stockholm (nơi có phòng trong mô phỏng của chúng ta).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Những điểm chính và Bước tiếp theo

### ✅ Những gì bạn đã học:

1. **Mẫu WorkflowBuilder**
   - Sử dụng `.set_start_executor()` để định nghĩa điểm vào
   - Sử dụng `.add_edge(from, to, condition=...)` cho định tuyến có điều kiện
   - Gọi `.build()` để hoàn thiện workflow

2. **Định tuyến có điều kiện**
   - Các hàm điều kiện kiểm tra `AgentExecutorResponse`
   - Phân tích các đầu ra cấu trúc để đưa ra quyết định định tuyến
   - Trả về `True` để kích hoạt một cạnh, `False` để bỏ qua nó

3. **Tích hợp công cụ**
   - Sử dụng `@ai_function` để chuyển đổi các hàm Python thành công cụ AI
   - Các agent tự động gọi công cụ khi cần thiết
   - Công cụ trả về JSON mà agent có thể phân tích

4. **Đầu ra có cấu trúc**
   - Sử dụng mô hình Pydantic để trích xuất dữ liệu an toàn theo kiểu
   - Đặt `response_format=MyModel` khi tạo agent
   - Phân tích phản hồi với `Model.model_validate_json()`

5. **Executor tùy chỉnh**
   - Sử dụng `@executor(id="...")` để tạo thành phần workflow
   - Executor có thể biến đổi dữ liệu hoặc thực hiện các tác động phụ
   - Sử dụng `ctx.yield_output()` để tạo ra kết quả workflow

### 🚀 Ứng dụng thực tế:

- **Đặt chuyến đi**: Kiểm tra khả năng, gợi ý các lựa chọn thay thế, so sánh phương án
- **Dịch vụ khách hàng**: Định tuyến dựa trên loại vấn đề, cảm xúc, ưu tiên
- **Thương mại điện tử**: Kiểm tra tồn kho, gợi ý thay thế, xử lý đơn hàng
- **Kiểm duyệt nội dung**: Định tuyến dựa trên điểm độc hại, báo cáo người dùng
- **Quy trình phê duyệt**: Định tuyến dựa trên số tiền, vai trò người dùng, mức độ rủi ro
- **Xử lý đa giai đoạn**: Định tuyến dựa trên chất lượng, độ đầy đủ của dữ liệu

### 📚 Bước tiếp theo:

- Thêm các điều kiện phức tạp hơn (nhiều tiêu chí)
- Triển khai vòng lặp với quản lý trạng thái workflow
- Thêm các workflow phụ cho các thành phần tái sử dụng
- Tích hợp với API thực tế (đặt phòng khách sạn, hệ thống tồn kho)
- Thêm xử lý lỗi và đường dẫn dự phòng
- Trực quan hóa workflow với công cụ trực quan tích hợp sẵn


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố miễn trừ trách nhiệm**:
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
